In [2]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from tqdm import tqdm
import pandas as pd
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

# Load the CSV file again
data = pd.read_csv('preprocessed_content_market.csv')

# Extracting each row from the 'preprocessed_content' column and storing it in a list
content_list = data['preprocessed_content'].tolist()

# Displaying the first few items in the list to verify
content_list[:2]

['리뷰 마켓 컬리 8 주년 위 크 역대 급 특 가 추천 상품 포도 계란 소고기 마트 보다 저렴하게 고 미리 레 마켓 컬리 8 주년 위 크 역대 급 특 가 추천 상품 포도 계란 소고기 마트 보다 저렴하게 안녕하세요 네이버 인 플루 언 서 고 미리 레 입니다 제 가 너무나 애정 하고 있는 마켓 컬리',
 '식품 리뷰 마켓 컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억 국민학교 떡볶이 드셔 보세요 코 나 맘 지인 추천 해 준 마켓 컬리 떡볶이 추억 국민학교 떡볶이 에요 추억 국민학교 떡볶이 는 냉동식품 으로 조리법 엄청 간단해요 떡 어묵 기름 에 한번 볶아서 하면']

In [3]:
# Kiwi와 KeyBERT 모델 초기화
kiwi = Kiwi()
kw_model = KeyBERT()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
            results.append(token)
    return results

# 각 행의 텍스트에 대해 명사 추출 및 키워드 추출
extracted_keywords = []
for text in tqdm(content_list):
    # 명사 추출
    nouns = noun_extractor(text)
    after_kiwi_text = ' '.join(nouns)
    
    # KeyBERT를 사용한 키워드 추출
    preprocessed_content_keywords = kw_model.extract_keywords(after_kiwi_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=3)
    
    # 추출된 키워드 저장
    extracted_keywords.append(preprocessed_content_keywords)

# 결과 출력
print(extracted_keywords[:5])


100%|██████████| 3993/3993 [00:51<00:00, 77.59it/s]

[[('컬리', 0.5002), ('소고기', 0.4581), ('애정', 0.4579)], [('냉동식품', 0.5079), ('분식집', 0.475), ('국민학교', 0.4662)], [('컬리', 0.5529), ('키트', 0.4939), ('유저', 0.4168)], [('컬리', 0.5353), ('어시장', 0.531), ('모듬', 0.4543)], [('충성', 0.3416), ('쿠폰', 0.333), ('컬리', 0.3109)]]


In [4]:
# Extracting only the words from the extracted keywords
words_only = [[keyword[0] for keyword in keywords_list] for keywords_list in extracted_keywords]

# Creating a new DataFrame with the words only
keywords_df = pd.DataFrame({
    'Extracted_Keywords': words_only
})

keywords_df.head()

,Extracted_Keywords
0,"[컬리, 소고기, 애정]"
1,"[냉동식품, 분식집, 국민학교]"
2,"[컬리, 키트, 유저]"
3,"[컬리, 어시장, 모듬]"
4,"[충성, 쿠폰, 컬리]"


In [10]:
importance_df = pd.DataFrame()
importance_df['importance'] = data['likes'].values + data['comments'].values
importance_df.head()

,importance
0,56.0
1,6.0
2,83.0
3,6.0
4,10.0


In [11]:
importance_keyword_df = pd.concat([importance_df,keywords_df],axis=1)
importance_keyword_df.head()

,importance,Extracted_Keywords
0,56.0,"[컬리, 소고기, 애정]"
1,6.0,"[냉동식품, 분식집, 국민학교]"
2,83.0,"[컬리, 키트, 유저]"
3,6.0,"[컬리, 어시장, 모듬]"
4,10.0,"[충성, 쿠폰, 컬리]"


In [15]:
keyword_hashtags_importance_df = pd.concat([data['hashtags'],importance_keyword_df], axis=1)
keyword_hashtags_importance_df.head()

,hashtags,importance,Extracted_Keywords
0,"[마켓컬리8주년위크, 마켓컬리, 컬리8주년, 마켓컬리할인, 마켓컬리추천템, 컬리추천...",56.0,"[컬리, 소고기, 애정]"
1,"[마켓컬리, 마켓컬리, 오늘도컬리, ID_tkdlqlek]",6.0,"[냉동식품, 분식집, 국민학교]"
2,"[마켓컬리추천템, 윅셔너리뱡뱡면, 뱡뱡면, 금미옥, 금미옥닭볶이, 마켓컬리, 자취생...",83.0,"[컬리, 키트, 유저]"
3,"[모듬조개탕, 조개탕밀키트, 마켓컬리조개탕, 마켓컬리추천, 땡초넣은모듬조개탕, 땡초...",6.0,"[컬리, 어시장, 모듬]"
4,"[마켓컬리, 홍루이젠, 도산분식, 포켓몬빵, 대구반할만떡, 와코사각유부, 홍루이젠,...",10.0,"[충성, 쿠폰, 컬리]"


In [22]:
keyword_hashtags_importance_df.to_csv('keyword_hashtags_importance_market.csv', encoding='utf-8', index=False)